This Program is to clean indonesian documents 

In [ ]:
import os
import shutil
import re
import spacy
import nltk
import statistics

from spacy.language import Language
from langdetect import detect, detect_langs
from spacy_langdetect import LanguageDetector
from spacy_language_detection import LanguageDetector

In [ ]:
def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)  # We use the seed 42

In [ ]:
directory = './test/txts/' #here are the original txt
save_dir = './test/txts/hasil/' #the outputs will be generated here

In [ ]:
txt_filenames = [filename for filename in os.listdir(directory) if filename.endswith('.txt')]
txt_filenames.sort()

In [ ]:
txt_filenames

In [ ]:
'''
list_of_chars = ['|']
pattern = '[' +  ''.join(list_of_chars) +  ']'

for filename in saved_filenames:
    filepath = os.path.join(directory, filename) 
    with open(filepath, 'r') as f:
        lines = []
        for line in f:
            line = line.strip()
            try:
                #lang = detect(line) 
                #if lang == 'id':
                lines.append(line) 
            except Exception:
                pass
                                           
        print(lines)
        

                            words = nltk.tokenize.word_tokenize(line)
                            #print(words)
                            add=0
                            for word in words:
                                word_langs = detect_langs(word)
                                print(word_langs)
                                for word_language in word_langs:
                                    if (word_language.lang == "id") or ((word_language.lang == "en") and (word_language.prob>0.7)):
                                        add=1
                                        continue
                                    
                            
                            if add==1:
                                lines.append(line) 
                                continue
                            else:
                                print(line) 
                                #print(word_langs) 
                                #pass    
                                ,,,  
        
                
                    doc = nlp_model(line) #3
                    detect_language = doc._.language #4
                    print(line, detect_language)
                    '''

In [ ]:
# ignores words containing alphabet characters with a length less than 2, unless they contain a number or are in list:
'''
def ignore_small_words(text):
    words = text.split()
    filtered_words = [word for word in words if len(word) >= 3 or any(char.isdigit() for char in word) or word.lower() in list_3_words]
    filtered_text = " ".join(filtered_words)
    return filtered_text
    '''

In [ ]:
#receives a text input and ignores the first or last word if it has a length less than 2:
def ignore_small_words_first_last(text):
    words = text.split()
    if (len(words) > 0 and (len(words[0]) <= 2 and (words[0] not in [".",","]))):
        words = words[1:]
    if (len(words) > 0 and (len(words[-1]) <= 2 and (words[-1] not in [".",","]))):
        words = words[:-1]
    filtered_text = " ".join(words)
    return filtered_text

In [ ]:
# The characters to search for and change to nothing
# Nyari karakter2 yang dihapus
list_of_chars = ['|','\xa0','£','»']
# Create the regular expression pattern by joining the search characters with the "or" operator
pattern = '[' +  ''.join(list_of_chars) +  ']'

# The strings to search for and then skip
# Detect paging
search_strings = [
                  "KONVENSI NASIONAL IAP - CIDES",
                 ]
# Create the regular expression pattern by joining the search strings with the "or" operator
pattern_strings = "|".join(search_strings)

nlp_model = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp_model.add_pipe('language_detector', last=True)


for filename in txt_filenames:
    filepath = os.path.join(directory, filename) 
    print(filepath)

    with open(filepath, 'r') as f:
        lines = []
        for line in f:
            #Ngapus character aneh
            line = re.sub(pattern, '', line.strip())
            #Ngapus kalau ada spasi berlebih
            line = re.sub(' +', ' ', line)
            #Ngapus kata pendek di depan dan belakang
            line = ignore_small_words_first_last(line)
            #Ngapus kata pendek
            #line = ignore_small_words(line)
            
            # Use the `search` function from the `re` module to find the search string in the text
            matches = re.findall(pattern_strings, line)
            if len(matches) > 0:
                continue
            else:
                try:
                    
                    langs = detect_langs(line)
                    add=0
                    for language in langs:
                        if (language.lang == "id") :
                            lines.append(line)
                            break
                        elif language.prob>0.4:
                            words = nltk.tokenize.word_tokenize(line)
                            average_length = statistics.mean(map(len,words))
                            if (average_length<3):
                                break
                            else:   
                                lines.append(line)
                                break
                        else: #Non Indonesian                                
                            print(line,langs)
                            continue

                except Exception:
                    pass
        
        occurrence = {item: lines.count(item) for item in lines}        
        occurrence = {key:val for key, val in occurrence.items() if val <=2}

        #print(occurrence)
        text = (" ".join(occurrence))
        text = text.replace(" . ", " ")
        text = re.sub(r'\.+', '.', text)
        with open(os.path.join(save_dir, filename), 'w') as f:
            f.write(text)
            del(text)
            f.close()
        #print(lines)
